## WICCI Downscaled Data: Precip - 20 year - seasonal (CMIP6 source)

Objectives
* aggregate data for prcp, e.g. mean, standard deviation, days over 1",2",3", for SSP126, SSP245, SSP370, and SSP585
* run for every 20 year model window, e.g. 2021-2040 for 2030 average
* calculate averages by WICCI seasons
* creat new netcdf file(s) for aggregate data for each 20-year timeframe
* calculate intermodel standard deviation differently -- find average of each model within the current time window and
  then calculate standard deviation across models

Eric Compas, compase@uww.edu 11/17/2021, 1/19/2022, 1/25/2022, 2/28/2022, 6/6/2022, 4/10/2025

In [1]:
import netCDF4
import numpy as np
import os
import datetime
import gc
from netCDF4 import Dataset,num2date,date2num

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
base_folder = r"X:\CMIP6_Data"
if not os.path.isdir(base_folder):
    print("Base folder not valid")

In [4]:
#out_folder = "Z:/Climate_Data/aggregate_files"
out_folder = r"C:\Users\compase\Dropbox\Spring_2025\WICCI\CMIP6_WICCI_Output"
#out_folder  = "/Users/ericcompas/Climate_Data/output"
if not os.path.isdir(out_folder):
    print("Out folder not valid")

The BIG LOOP.

Look across all years, models, and create aggregate file for each year.

In [ ]:
# loop across models
models = ["ssp126","ssp245","ssp370","ssp585"]
#models = ["ssp245"]
for m in models:
    print("Processing climate scenario "+m)

    # get subfolders/GCMs for current model
    GCMs = os.listdir(os.path.join(base_folder,m))
    #GCMs = GCMs[:5]  # for testing only

    # loop across years in 20-year increments
    for y in range(2030,2091,20):    # 2030,2091,20
        print("  Processing base year "+str(y))

        # get mean and std for prcp for current 20-year window
        # loop through GCMS first and then years in current window
        # (create large blank masked arrays more memory efficient?)
        num_models = len(GCMs)
        years = 20
        realizations = 3
        #prcp_all_array = np.ma.empty([num_models*years*realizations, 365, 89, 91])
        prcp_all_array = np.ma.empty([num_models*years*realizations, 365, 89, 91], dtype=np.float32)
        prcp_means_array = np.ma.empty([num_models, 365, 89, 91])
        
         # indices for adding to blank arrays
        i = 0
        j = 0
        
        for gcm in GCMs:      
            print("    Processing GCM "+gcm)
            prcp_gcm_array = []
            for yr in range(y-9,y+11):
                print("      Processing year "+str(yr))
                realizations = ["01","02","03"]
                for r in realizations:
                    nf = os.path.join(base_folder,m,gcm,"r1i1p1","prcp_"+r+"_"+str(yr)+".nc")
                    try:
                        n = netCDF4.Dataset(nf)
                    except:
                        #print("File not found: "+nf)
                        q = 0
                    else:
                        prcp = n.variables['prcp']
                        
                        # convert to inches
                        prcp = prcp[:,:,:] * 0.0393701
                        
                        # get mask - to reapply below
                        mask = np.ma.getmask(prcp[0])
                        
                        # handle leap years -- average Feb 28 and 29
                        if prcp.shape[0] == 366:
                            # get and average Feb 28 and 29 for prcp
                            prcp_leap = prcp[58:60,:,:]
                            prcp_mean_leap = np.ma.mean(prcp_leap,axis=0)
                            prcp_part1 = np.ma.append(prcp[0:58,:,:],np.ma.expand_dims(prcp_mean_leap,axis=0),axis=0)
                            prcp = np.ma.append(prcp_part1,prcp[60:367,:,:],axis=0)
                        
                        # add prcp array for gcm, year, realization to total array
                        prcp_all_array[i] = prcp[:,:,:]
                        
                        # add prcp array for gcm, year, realization to gcm-specific array
                        prcp_gcm_array.append(prcp[:,:,:])
                        
                        i += 1

            # calculate mean precip for gcm over time window (for std calculation)
            prcp_model_mean = np.ma.mean(prcp_gcm_array,axis=0)

            # add this to an array of means (one for each gcm)
            prcp_means_array[j] = prcp_model_mean[:,:,:]
            
            j += 1

        ##################################################
        ## calculate metrics for current 20-year window ##
        ##################################################
        
        # calculate daily mean from total array    
        prcp_mean = np.ma.mean(prcp_all_array,axis=0)

        # calculate seasonly mean from total array
        prcp_seasons_total = np.ma.empty([4, 89, 91])
        prcp_seasons_std = np.ma.empty([4, 89, 91])
        prcp_seasons_gt1 = np.ma.empty([4, 89, 91])
        prcp_seasons_gt2 = np.ma.empty([4, 89, 91])
        prcp_seasons_gt3= np.ma.empty([4, 89, 91])
        prcp_seasons_gt4 = np.ma.empty([4, 89, 91])
        prcp_seasons_gt5 = np.ma.empty([4, 89, 91])
        prcp_seasons_norain = np.ma.empty([4, 89, 91])
        
        # index for julian/ordinal calendar (start/end of each season)
        julian = [0,59,151,243,334,366]
        
        # loop through seasons
        for season in range(1,5):
            # calculate season precip total for each season
            if season == 1:  # winter
                part1 = prcp_mean[(julian[season-1]):(julian[season])]
                part2 = prcp_mean[(julian[4]):(julian[5])]
                prcp_season_total = np.ma.sum(np.ma.append(part1,part2,axis=0),axis=0)
            else:
                prcp_season_total = np.ma.sum(prcp_mean[(julian[season-1]):(julian[season])],axis=0)
            prcp_seasons_total[season-1] = prcp_season_total
            
            # calculate inter-model standard deviation
            all_std_nparray = np.array(prcp_means_array)
            if season == 1: # winter
                part1 = all_std_nparray[:,(julian[season-1]):(julian[season]),:,:]
                part2 = all_std_nparray[:,(julian[4]):(julian[5]),:,:]
                season_std_nparray = np.ma.append(part1,part2,axis=1)
            else:
                season_std_nparray = all_std_nparray[:,(julian[season-1]):(julian[season]),:,:]
            # loop over each model -- add to total seasonly sample array
            for i in range(0, (np.shape(all_std_nparray)[0])):
                model_array = season_std_nparray[i]
                if i==0:
                    concat_array = model_array
                else:
                    concat_array = np.ma.concatenate((concat_array,model_array),axis=0)
            std_season = np.ma.std(concat_array,axis=0)
            prcp_seasons_std[season-1] = np.ma.masked_array(std_season,mask)
            
            # calc extreme estimates - prcp
            all_nparray = np.array(prcp_all_array)
            if season == 1: # winter
                part1 = all_nparray[:,(julian[season-1]):(julian[season]),:,:]
                part2 = all_nparray[:,(julian[4]):(julian[5]),:,:]
                season_nparray = np.ma.append(part1,part2,axis=1)
            else:
                season_nparray = all_nparray[:,(julian[season-1]):(julian[season]),:,:]
            # get number of days in season and number of "samples"(needed for estimate formula)
            num_days = np.shape(season_nparray)[1]
            num_samples = np.shape(season_nparray)[0]*np.shape(season_nparray)[1]     
            
            # prcp extreme - prcp gt 1" (25.4 mm)
            # get number of days meeting threshold (by iteration)
            extreme_sum_days = (season_nparray >= 1).sum(axis=0)
            # add number of days (by season)
            extreme_sum = extreme_sum_days.sum(axis=0)
            # estimate of number of seasonly extreme days
            extreme_days = (extreme_sum / num_samples) * num_days
            # add to array
            prcp_seasons_gt1[season-1] = np.ma.masked_array(extreme_days,mask)
            
            # prcp extreme - prcp gt 2" (50.8 mm)     
            # get number of days meeting threshold (by iteration)
            extreme_sum_days = (season_nparray >= 2).sum(axis=0)
            # add number of days (by season)
            extreme_sum = extreme_sum_days.sum(axis=0)
            # estimate of number of seasonly extreme days
            extreme_days = (extreme_sum / num_samples) * num_days
            # add to array
            prcp_seasons_gt2[season-1] = np.ma.masked_array(extreme_days,mask)
            
            # prcp extreme - prcp gt 3" (76.2 mm)     
            # get number of days meeting threshold (by iteration)
            extreme_sum_days = (season_nparray >= 3).sum(axis=0)
            # add number of days (by season)
            extreme_sum = extreme_sum_days.sum(axis=0)
            # estimate of number of seasonly extreme days
            extreme_days = (extreme_sum / num_samples) * num_days
            # add to array
            prcp_seasons_gt3[season-1] = np.ma.masked_array(extreme_days,mask)

            # prcp extreme - prcp gt 4" (101.6 mm)     
            # get number of days meeting threshold (by iteration)
            extreme_sum_days = (season_nparray >= 4).sum(axis=0)
            # add number of days (by season)
            extreme_sum = extreme_sum_days.sum(axis=0)
            # estimate of number of seasonly extreme days
            extreme_days = (extreme_sum / num_samples) * num_days
            # add to array
            prcp_seasons_gt4[season-1] = np.ma.masked_array(extreme_days,mask)

            # prcp extreme - prcp gt 5" (127.0 mm)     
            # get number of days meeting threshold (by iteration)
            extreme_sum_days = (season_nparray >= 5).sum(axis=0)
            # add number of days (by season)
            extreme_sum = extreme_sum_days.sum(axis=0)
            # estimate of number of seasonly extreme days
            extreme_days = (extreme_sum / num_samples) * num_days
            # add to array
            prcp_seasons_gt5[season-1] = np.ma.masked_array(extreme_days,mask)
            
            # prcp extreme(?) - prcp is zero    
            # get number of days meeting threshold (by iteration)
            extreme_sum_days = (season_nparray == 0).sum(axis=0)
            # add number of days (by season)
            extreme_sum = extreme_sum_days.sum(axis=0)
            # estimate of number of seasonly extreme days
            extreme_days = (extreme_sum / num_samples) * num_days
            # add to array
            prcp_seasons_norain[season-1] = np.ma.masked_array(extreme_days,mask)

        # write netcdf files with results
        filename = "prcp_"+m+"_"+str(y)+"_20yr_seasonal.nc"
        newfile = os.path.join(out_folder,filename)
        ncfile = netCDF4.Dataset(newfile,mode='w',format='NETCDF4_CLASSIC')
        lat_dim = ncfile.createDimension('lat', 89)     # latitude axis
        lon_dim = ncfile.createDimension('lon', 91)    # longitude axis
        time_dim = ncfile.createDimension('time', None) # unlimited axis (can be appended to).

        ncfile.title='Aggregate seasonal prcp values for WICCI downscaled climate data (CMIP6) for all GCMs for '+m+' and 20-year window around year '+str(y)
        ncfile.subtitle="Data source: UW-Madison WICCI; Data aggregation: Eric Compas, compase@uww.edu"
        lat = ncfile.createVariable('lat', np.float64, ('lat',))
        lat.units = 'degrees_north'
        lat.long_name = 'latitude'
        lon = ncfile.createVariable('lon', np.float64, ('lon',))
        lon.units = 'degrees_east'
        lon.long_name = 'longitude'
        time = ncfile.createVariable('time', np.float64, ('time',))
        timeunits = 'days since '+str(y)+'-01-01'
        time.units = timeunits
        time.long_name = 'time'

        prcp_total = ncfile.createVariable('prcp_total',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        prcp_total.units = 'inches' # inches
        prcp_total.standard_name = 'total of mean of daily precipitation (inches) per month across 20-year window'
        prcp_total.missing_value = -32768
        
        prcp_std = ncfile.createVariable('prcp_std',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        prcp_std.units = 'inches' # inches
        prcp_std.standard_name = 'standard deviation of daily precipitation (inches) per month across 20-year window'
        prcp_std.missing_value = -32768
        
        prcp_gt1 = ncfile.createVariable('prcp_gt1',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        prcp_gt1.units = 'days' # number of days
        prcp_gt1.standard_name = 'estimated number of days 1" or more of rain (25.4 mm) per month across 20-year window'
        prcp_gt1.missing_value = -32768
        
        prcp_gt2 = ncfile.createVariable('prcp_gt2',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        prcp_gt2.units = 'days' # number of days
        prcp_gt2.standard_name = 'estimated number of days 2" or more of rain (50.8 mm) per month across 20-year window'
        prcp_gt2.missing_value = -32768
        
        prcp_gt3 = ncfile.createVariable('prcp_gt3',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        prcp_gt3.units = 'days' # number of days
        prcp_gt3.standard_name = 'estimated number of days 3" or more of rain (76.2 mm) per month across 20-year window'
        prcp_gt3.missing_value = -32768

        prcp_gt4 = ncfile.createVariable('prcp_gt4',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        prcp_gt4.units = 'days' # number of days
        prcp_gt4.standard_name = 'estimated number of days 4" or more of rain (101.6 mm) per season across 30-year window'
        prcp_gt4.missing_value = -32768

        prcp_gt5 = ncfile.createVariable('prcp_gt5',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        prcp_gt5.units = 'days' # number of days
        prcp_gt5.standard_name = 'estimated number of days 5" or more of rain (127 mm) per season across 30-year window'
        prcp_gt5.missing_value = -32768
        
        prcp_norain = ncfile.createVariable('prcp_norain',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        prcp_norain.units = 'days' # number of days
        prcp_norain.standard_name = 'estimated number of days with no rain (0 mm) per month across 20-year window'
        prcp_norain.missing_value = -32768 

        # Write latitudes, longitudes
        # Note: the ":" is necessary in these "write" statements
        n_lat = n.variables['lat']
        n_lon = n.variables['lon']
        lat[:] = n_lat[:]
        lon[:] = n_lon[:]

        # write temp variables
        prcp_total[:,:,:] = prcp_seasons_total
        prcp_std[:,:,:] = prcp_seasons_std
        prcp_gt1[:,:,:] = prcp_seasons_gt1
        prcp_gt2[:,:,:] = prcp_seasons_gt2
        prcp_gt3[:,:,:] = prcp_seasons_gt3
        prcp_gt4[:,:,:] = prcp_seasons_gt4
        prcp_gt5[:,:,:] = prcp_seasons_gt5
        prcp_norain[:,:,:] = prcp_seasons_norain

        # write time
        yystart = y
        print("Writing time. Year = "+str(y))
        # Create datetime objects for the middle of each season
        # Winter (Jan 15), Spring (Apr 15), Summer (Jul 15), Fall (Oct 15)
        season_months = [1, 4, 7, 10]  # Middle month of each season
        datesout = [datetime.datetime(y, month, 15, 0) for month in season_months]
        time[:] = date2num(datesout, timeunits)

        # close file
        ncfile.close()

        # report progress
        print("Wrote: "+filename)
        
        
        # clear some memory, invoke Python garbage collector
        del prcp_all_array
        del prcp_means_array
        del prcp_seasons_total
        del prcp_seasons_std
        del prcp_seasons_gt1
        del prcp_seasons_gt2
        del prcp_seasons_gt3
        del prcp_seasons_gt4
        del prcp_seasons_gt5
        del prcp_seasons_norain
        del prcp_mean
        gc.collect()
        print("Cleared some memory up...restarting loop")
        

Processing climate scenario ssp126
  Processing base year 2030
    Processing GCM ACCESS-CM2
      Processing year 2021
      Processing year 2022
      Processing year 2023
      Processing year 2024
      Processing year 2025
      Processing year 2026
      Processing year 2027
      Processing year 2028
      Processing year 2029
      Processing year 2030
      Processing year 2031
      Processing year 2032
      Processing year 2033
      Processing year 2034
      Processing year 2035
      Processing year 2036
      Processing year 2037
      Processing year 2038
      Processing year 2039
      Processing year 2040
    Processing GCM CanESM5
      Processing year 2021
      Processing year 2022
      Processing year 2023
      Processing year 2024
      Processing year 2025
      Processing year 2026
      Processing year 2027
      Processing year 2028
      Processing year 2029
      Processing year 2030
      Processing year 2031
      Processing year 2032
      Processing